In [1]:
import lightgbm as lgb, catboost as cb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)

# Prepare data

In [2]:
df = pd.read_csv('train.csv').sort_values('timestamp')
df[df.loc[:, df.dtypes == object].columns] = df.loc[:, df.dtypes == object].astype('category')
df.year_of_construction = df.year_of_construction.astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27232 entries, 0 to 27231
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   district_area                      27232 non-null  float64 
 1   road_distance_1                    27232 non-null  float64 
 2   road_distance_2                    27232 non-null  float64 
 3   year_of_construction               14314 non-null  category
 4   bulvar_ring_km                     27232 non-null  float64 
 5   bus_station_distance               27232 non-null  float64 
 6   cafe_count                         27232 non-null  int64   
 7   fitness_center_distance            27232 non-null  float64 
 8   floor                              27065 non-null  float64 
 9   district_population                27232 non-null  int64   
 10  total_area                         27232 non-null  int64   
 11  green_part                         27232 non-n

In [3]:
df['res_m3'] = df.result_price/df.total_area

In [4]:
df[((df.res_m3<22000)|(df.res_m3>400000))]

,district_area,road_distance_1,road_distance_2,year_of_construction,bulvar_ring_km,bus_station_distance,cafe_count,fitness_center_distance,floor,district_population,total_area,green_part,green_zone_distance,healthcare_centers_count,id,kitchen_area,kremlin_distance,leisure_count,living_area,market_count,wall_material,floors_num,metro_minutes,mkad_distance,rooms_num,office_count,park_distance,result_price,product_type,public_transport_station_distance,sadovoe_km,base_school_distance,sport_count,state,district_name,timestamp,malls_count,railway_station_distance,res_m3
50,8.287309e+06,3.170454,4.782849,NaN,14.126353,3.681055,139,1.069785,1.0,145576,51,16.78,0.395952,3,51,NaN,14.619975,3,33.0,9,NaN,NaN,6.737871,4.861966,NaN,17,0.599852,1000000,Investment,1.409641,12.456588,0.478507,53,NaN,Caricyno,2011-10-03,44,14.403918,1.960784e+04
59,6.879020e+06,1.016291,3.249057,NaN,14.974261,5.886793,97,1.257800,14.0,111874,64,32.24,0.221275,0,60,NaN,15.618234,0,44.0,10,NaN,NaN,14.336688,3.249057,NaN,14,2.458768,990000,Investment,0.775311,13.567695,0.198752,40,NaN,Chertanovo Central'noe,2011-10-04,40,16.027230,1.546875e+04
66,1.675112e+07,0.541867,1.593603,NaN,13.718306,6.210762,119,0.178748,9.0,156377,50,22.70,0.086103,1,67,NaN,14.884413,0,29.0,3,NaN,NaN,5.216032,0.541867,NaN,22,1.294504,1000000,Investment,0.940464,12.879286,0.263759,52,NaN,Strogino,2011-10-05,24,16.954525,2.000000e+04
81,2.615514e+07,2.640847,4.180915,NaN,22.956078,6.884292,25,0.328024,9.0,102618,50,24.20,0.268879,1,82,NaN,23.638102,0,30.0,3,NaN,NaN,12.329093,4.998718,NaN,4,1.804629,990000,Investment,2.907820,21.622904,0.754134,19,NaN,Juzhnoe Butovo,2011-10-07,14,25.174203,1.980000e+04
83,8.398461e+06,1.018012,2.694403,NaN,4.015050,4.898193,828,0.296969,7.0,28179,66,18.07,0.639098,4,84,NaN,5.798406,34,42.0,11,NaN,NaN,12.441523,8.624305,NaN,287,2.765772,990000,Investment,4.037727,3.272049,0.937935,134,NaN,Basmannoe,2011-10-07,70,4.849217,1.500000e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26925,5.299528e+07,0.464679,9.152001,1.0,28.385104,15.166222,7,5.980372,17.0,21819,1,39.40,0.991824,0,26928,1.0,29.133765,0,1.0,0,1.0,1.0,144.236338,11.751277,1.0,0,9.684197,3793778,OwnerOccupier,21.494546,27.271396,7.671322,0,1.0,Poselenie Desjonovskoe,2014-12-20,0,30.195354,3.793778e+06
27069,2.615514e+07,4.374199,5.290220,2004.0,23.614636,8.274640,18,0.276503,7.0,102618,74,23.89,0.000000,1,27072,10.0,24.316814,0,45.0,0,1.0,17.0,2.769421,5.954171,3.0,0,2.574492,1000000,Investment,1.390257,22.333634,0.458845,15,3.0,Juzhnoe Butovo,2014-12-23,8,27.403166,1.351351e+04
27046,1.175477e+07,4.826458,6.230198,1985.0,12.243011,6.455786,135,0.599670,3.0,165727,60,18.57,0.125704,5,27049,10.0,12.985410,6,34.0,10,1.0,17.0,17.293340,4.826458,2.0,8,0.823331,1000000,Investment,1.744813,10.752819,0.303131,61,2.0,Mar'ino,2014-12-23,39,15.263185,1.666667e+04
27134,7.128794e+06,1.318921,1.318921,1977.0,16.677132,1.964177,85,0.466904,2.0,129207,60,17.46,0.741505,1,27137,8.0,17.291262,2,39.0,9,1.0,9.0,13.709829,1.320650,3.0,2,3.010594,1000000,Investment,2.901894,15.036554,0.099743,40,3.0,Orehovo-Borisovo Juzhnoe,2014-12-25,30,17.645663,1.666667e+04


In [5]:
values=['Poselenie Kievskij',
 'Poselenie Mihajlovo-Jarcevskoe',
 'Poselenie Klenovskoe',
 'Poselenie Shhapovskoe',
 'Poselenie Marushkinskoe',
 'Vostochnoe']
df = df[df.district_name.isin (values) == False ]

In [6]:
ind = df[(df.kitchen_area + df.living_area > 2 * df.total_area)].index

In [7]:
ind = ind.append(df[((df.res_m3<22000)|(df.res_m3>400000))].index)

In [8]:
df = df.drop (index=ind, axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26678 entries, 0 to 27231
Data columns (total 39 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   district_area                      26678 non-null  float64 
 1   road_distance_1                    26678 non-null  float64 
 2   road_distance_2                    26678 non-null  float64 
 3   year_of_construction               14014 non-null  category
 4   bulvar_ring_km                     26678 non-null  float64 
 5   bus_station_distance               26678 non-null  float64 
 6   cafe_count                         26678 non-null  int64   
 7   fitness_center_distance            26678 non-null  float64 
 8   floor                              26512 non-null  float64 
 9   district_population                26678 non-null  int64   
 10  total_area                         26678 non-null  int64   
 11  green_part                         26678 non-n

In [9]:
df[~df.year_of_construction.isnull()].head()

,district_area,road_distance_1,road_distance_2,year_of_construction,bulvar_ring_km,bus_station_distance,cafe_count,fitness_center_distance,floor,district_population,total_area,green_part,green_zone_distance,healthcare_centers_count,id,kitchen_area,kremlin_distance,leisure_count,living_area,market_count,wall_material,floors_num,metro_minutes,mkad_distance,rooms_num,office_count,park_distance,result_price,product_type,public_transport_station_distance,sadovoe_km,base_school_distance,sport_count,state,district_name,timestamp,malls_count,railway_station_distance,res_m3
8135,5.646405e+06,2.116263,2.169200,1980.0,13.854330,21.354034,142,0.821462,10.0,68630,53,15.44,0.529494,3,8138,8.0,15.345902,0,30.0,5,1.0,16.0,18.420277,2.169200,2.0,25,0.875814,9000000,Investment,2.264785,13.270117,0.364994,46,3.0,Lianozovo,2013-05-25,48,13.396969,169811.320755
8154,8.889467e+06,0.833039,1.463849,1970.0,20.077081,1.527929,31,0.147876,6.0,78616,45,28.73,0.240982,2,8157,6.0,20.728839,0,27.0,3,1.0,9.0,9.928954,1.704870,2.0,3,2.590344,7100000,Investment,2.578740,18.682566,0.183503,19,3.0,Severnoe Butovo,2013-05-27,25,20.961121,157777.777778
8153,2.149409e+07,4.586082,5.602182,2014.0,25.032110,10.125933,18,0.822324,2.0,9553,77,17.74,0.456704,0,8156,12.0,25.735256,0,41.0,1,6.0,17.0,29.242588,7.371716,3.0,0,1.773759,7011550,OwnerOccupier,5.610908,23.753388,0.936324,12,1.0,Poselenie Voskresenskoe,2013-05-27,5,29.254460,91059.090909
8186,1.139168e+07,1.905125,2.657336,2013.0,19.272537,7.253743,16,2.493236,17.0,247469,63,4.46,0.017572,0,8189,1.0,20.549464,0,NaN,1,4.0,17.0,45.322032,5.946908,2.0,1,4.616176,5903808,OwnerOccupier,0.431838,18.418929,0.214197,6,1.0,Nekrasovka,2013-05-28,4,24.061214,93711.238095
8175,9.629358e+06,1.080901,1.160352,1982.0,7.519704,11.944556,285,0.520086,15.0,112804,38,20.98,0.038049,3,8178,8.0,8.569880,7,20.0,4,1.0,16.0,52.407275,6.856666,1.0,83,1.755323,6450000,Investment,1.932385,6.651871,0.782119,95,NaN,Filevskij Park,2013-05-28,34,9.870472,169736.842105


# Validate

In [10]:
n_val = 1500
X_tr, y_tr = df.iloc[:-n_val].drop(columns=['id', 'timestamp', 'result_price','res_m3']), df.iloc[:-n_val].result_price
X_val, y_val = df.iloc[-n_val:].drop(columns=['id', 'timestamp', 'result_price','res_m3']), df.iloc[-n_val:].result_price
X, y = df.drop(columns=['id', 'timestamp', 'result_price','res_m3']), df.result_price
category_feat = X.loc[:, X.dtypes == 'category'].columns.to_list()

In [11]:
import lightgbm as lgb

tr_data = lgb.Dataset(X_tr, label=y_tr, free_raw_data=False,categorical_feature=category_feat )

model = lgb.train({'eta': 0.13, 'max_depth': -1,'max_cat_to_onehot':3}, tr_data, num_boost_round=300)
# 'zero_as_missing':True

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013637 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5752
[LightGBM] [Info] Number of data points in the train set: 25178, number of used features: 35
[LightGBM] [Info] Start training from score 7003797.923346


In [12]:
import numpy as np

y_pred_val = model.predict(X_val)
y_val_diff = y_pred_val - y_val

In [13]:
score = np.sqrt(mean_squared_log_error(y_val, y_pred_val))
score

0.3813518372963428

In [14]:
res = pd.DataFrame({'true':y_val, 'pred':y_pred_val, 'diff':(y_val-y_pred_val), 'err':(abs(y_val-y_pred_val)/y_val)} )

In [15]:
a=pd.concat([res, X_val], axis = 1)

In [16]:
a['res_m3'] = a.true/a.total_area

In [17]:
a[(a.err>0.5)].head()

,true,pred,diff,err,district_area,road_distance_1,road_distance_2,year_of_construction,bulvar_ring_km,bus_station_distance,cafe_count,fitness_center_distance,floor,district_population,total_area,green_part,green_zone_distance,healthcare_centers_count,kitchen_area,kremlin_distance,leisure_count,living_area,market_count,wall_material,floors_num,metro_minutes,mkad_distance,rooms_num,office_count,park_distance,product_type,public_transport_station_distance,sadovoe_km,base_school_distance,sport_count,state,district_name,malls_count,railway_station_distance,res_m3
25739,25750000,9.372413e+06,1.637759e+07,0.636023,1.308980e+07,1.930468,3.388559,2010.0,10.826196,3.031310,290,0.180765,15.0,178473,101,19.48,0.228939,4,13.0,12.036334,3,58.0,12,4.0,28.0,31.264427,4.381810,3.0,69,0.246976,Investment,3.740137,10.014778,1.472291,92,1.0,Pokrovskoe Streshnevo,46,10.521011,254950.495050
25736,2000000,7.945304e+06,-5.945304e+06,2.972652,1.675112e+07,0.778144,1.213140,1981.0,13.484522,7.537778,120,0.524518,2.0,156377,49,23.68,0.086649,1,7.0,14.641870,0,29.0,3,1.0,12.0,15.635535,0.778144,2.0,22,1.823553,Investment,1.025750,12.645176,0.383467,54,3.0,Strogino,26,19.101313,40816.326531
25767,2000000,5.131926e+06,-3.131926e+06,1.565963,5.027634e+06,0.379324,0.620480,1967.0,5.552437,5.105843,585,0.570906,1.0,155572,32,15.63,0.165455,1,1.0,7.026195,10,17.0,11,5.0,12.0,16.266509,9.793237,1.0,222,1.185106,Investment,1.114131,4.934251,0.229961,119,2.0,Butyrskoe,40,3.319152,62500.000000
25734,3500000,7.322946e+06,-3.822946e+06,1.092270,3.292112e+06,0.147625,4.035646,2013.0,11.915184,12.972124,190,0.778771,14.0,41504,54,15.43,0.138152,1,8.0,13.387571,0,27.0,8,4.0,25.0,9.564383,4.035646,2.0,44,1.973682,Investment,2.157098,11.294351,0.235781,69,2.0,Beskudnikovskoe,49,10.126758,64814.814815
25694,2000000,5.709601e+06,-3.709601e+06,1.854800,8.464344e+06,1.030213,1.347586,1973.0,17.609302,6.102950,61,1.296979,8.0,145088,45,16.21,0.272368,0,6.0,18.172890,2,30.0,5,1.0,12.0,54.169112,1.347586,2.0,7,2.274774,Investment,1.144223,16.072361,0.128968,25,2.0,Birjulevo Zapadnoe,31,18.450547,44444.444444


In [18]:
a[(a.true<1000001)&(a.err>1)].head()

,true,pred,diff,err,district_area,road_distance_1,road_distance_2,year_of_construction,bulvar_ring_km,bus_station_distance,cafe_count,fitness_center_distance,floor,district_population,total_area,green_part,green_zone_distance,healthcare_centers_count,kitchen_area,kremlin_distance,leisure_count,living_area,market_count,wall_material,floors_num,metro_minutes,mkad_distance,rooms_num,office_count,park_distance,product_type,public_transport_station_distance,sadovoe_km,base_school_distance,sport_count,state,district_name,malls_count,railway_station_distance,res_m3
25809,1000000,6.379171e+06,-5.379171e+06,5.379171,4.800968e+06,2.092862,2.100735,1962.0,2.174549,2.727032,1980,0.401547,9.0,102726,37,9.22,0.285280,1,7.0,2.813835,76,22.0,12,5.0,9.0,11.369179,12.725399,2.0,530,0.821564,Investment,1.980882,0.804303,0.753327,156,3.0,Jakimanka,83,2.803741,27027.027027
25886,1000000,4.751432e+06,-3.751432e+06,3.751432,4.808270e+06,0.567796,3.262022,1962.0,8.425011,6.043652,109,0.608355,5.0,101982,31,14.64,0.024842,1,5.0,9.469143,4,19.0,13,1.0,9.0,17.542310,5.136317,1.0,33,0.703712,Investment,2.804908,7.356242,0.299074,61,2.0,Tekstil'shhiki,34,9.864488,32258.064516
25995,1000000,6.739132e+06,-5.739132e+06,5.739132,1.050803e+07,1.137698,1.137879,1990.0,5.337137,4.467279,701,1.080235,13.0,57107,37,13.97,0.068209,2,8.0,6.729426,18,18.0,13,1.0,25.0,23.824438,10.159353,1.0,258,1.536835,Investment,1.650010,4.662721,0.157321,121,4.0,Timirjazevskoe,44,3.504058,27027.027027
26021,1000000,8.177718e+06,-7.177718e+06,7.177718,5.540726e+06,0.129026,0.420178,1958.0,4.439911,6.034482,908,1.224302,7.0,106445,35,9.76,0.044949,0,5.0,5.173259,35,25.0,8,5.0,8.0,13.725607,10.406129,2.0,290,2.378539,Investment,0.818880,3.318407,0.335803,122,2.0,Gagarinskoe,47,6.111192,28571.428571
25965,1000000,5.847896e+06,-4.847896e+06,4.847896,1.686153e+07,0.412076,0.445533,2006.0,13.404075,16.402635,56,0.950016,14.0,142462,38,17.38,0.475386,1,8.0,14.280519,2,17.0,0,1.0,14.0,56.376459,0.412076,1.0,17,1.154814,Investment,2.832287,12.340822,0.291413,32,2.0,Mozhajskoe,22,12.720103,26315.789474


In [19]:
pd.DataFrame({'featchure':X_tr.columns, 'imp': model.feature_importance()})

,featchure,imp
0,district_area,103
1,road_distance_1,274
2,road_distance_2,204
3,year_of_construction,736
4,bulvar_ring_km,90
5,bus_station_distance,184
6,cafe_count,139
7,fitness_center_distance,363
8,floor,439
9,district_population,90


# Train all

In [20]:
data = lgb.Dataset(X, label=y, free_raw_data=False,categorical_feature=category_feat )

model = lgb.train({'eta': 0.13, 'max_depth': -1,'max_cat_to_onehot':3,'zero_as_missing':True}, data, num_boost_round=350)
# 'zero_as_missing':True

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012304 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5750
[LightGBM] [Info] Number of data points in the train set: 26678, number of used features: 35
[LightGBM] [Info] Start training from score 7046184.222243


# TEST

In [21]:
df_test = pd.read_csv('test.csv', sep=',')
print(len(df_test))

3239


In [22]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3239 entries, 0 to 3238
Data columns (total 37 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   district_area                      3239 non-null   float64
 1   road_distance_1                    3239 non-null   float64
 2   road_distance_2                    3239 non-null   float64
 3   year_of_construction               2552 non-null   float64
 4   bulvar_ring_km                     3239 non-null   float64
 5   bus_station_distance               3239 non-null   float64
 6   cafe_count                         3239 non-null   int64  
 7   fitness_center_distance            3239 non-null   float64
 8   floor                              3239 non-null   float64
 9   district_population                3239 non-null   int64  
 10  total_area                         3239 non-null   int64  
 11  green_part                         3239 non-null   float

In [23]:
df_test['timestamp'] = pd.to_datetime(df_test['timestamp'])
df_test.timestamp = df_test.timestamp.values.astype(np.int64) // 10 ** 9
df_test[df_test.loc[:, df_test.dtypes == object].columns] = df_test.loc[:, df_test.dtypes == object].astype('category')
df_test.year_of_construction = df_test.year_of_construction.astype('category')

In [24]:
df_test = df_test.drop(columns=['id', 'timestamp'], axis=1)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3239 entries, 0 to 3238
Data columns (total 35 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   district_area                      3239 non-null   float64 
 1   road_distance_1                    3239 non-null   float64 
 2   road_distance_2                    3239 non-null   float64 
 3   year_of_construction               2552 non-null   category
 4   bulvar_ring_km                     3239 non-null   float64 
 5   bus_station_distance               3239 non-null   float64 
 6   cafe_count                         3239 non-null   int64   
 7   fitness_center_distance            3239 non-null   float64 
 8   floor                              3239 non-null   float64 
 9   district_population                3239 non-null   int64   
 10  total_area                         3239 non-null   int64   
 11  green_part                         3239 non

In [25]:
y_pred = model.predict(df_test)

In [26]:
df_sub = pd.read_csv('sample_submission.csv')
df_sub.result_price=y_pred
df_sub.to_csv('submission.csv', index=False)

In [27]:
df_sub

,id,result_price
0,27235,4.986341e+06
1,27236,4.597908e+06
2,27237,8.622306e+06
3,27238,5.229582e+06
4,27239,9.004785e+06
...,...,...
3234,30469,6.956423e+06
3235,30470,1.899821e+07
3236,30471,5.292265e+06
3237,30472,1.001193e+07


In [28]:
# !kaggle competitions submit -c aim2023-guess-the-price -f submission.csv -m "Message"